In [1]:
import os

FPS = 100
SCREENSIZE = (640, 480)
Imagees = {
    'rabbit': os.path.join(os.getcwd(), 'input/HouseDefend/images/dude.png'),
    'grass': os.path.join(os.getcwd(),  'input/HouseDefend/images/grass.png'),
    'castle': os.path.join(os.getcwd(), 'input/HouseDefend/images/castle.png'),
    'arrow': os.path.join(os.getcwd(),  'input/HouseDefend/images/bullet.png'),
    'badguy': os.path.join(os.getcwd(), 'input/HouseDefend/images/badguy.png'),
    'healthbar': os.path.join(os.getcwd(), 'input/HouseDefend/images/healthbar.png'),
    'health': os.path.join(os.getcwd(), 'input/HouseDefend/images/health.png'),
    'gameover': os.path.join(os.getcwd(),'input/HouseDefend/images/gameover.png'),
    'youwin': os.path.join(os.getcwd(), 'input/HouseDefend/images/youwin.png')
}

Sounds = {
    'hit': os.path.join(os.getcwd(), 'input/HouseDefend/audio/explode.wav'),
    'enemy': os.path.join(os.getcwd(), 'input/HouseDefend/audio/enemy.wav'),
    'shoot': os.path.join(os.getcwd(), 'input/HouseDefend/audio/shoot.wav'),
    'moonlight': os.path.join(os.getcwd(), 'input/HouseDefend/audio/moonlight.wav')
}

In [2]:
import sys
import math
import random
import pygame
from modules import *

def initGame():

    pygame.init()
    pygame.mixer.init()
    screen = pygame.display.set_mode(SCREENSIZE)
    pygame.display.set_caption('HouseDefend')

    game_images = {}
    for key, value in Imagees.items():
        game_images[key] = pygame.image.load(value)
    game_sounds = {}
    for key, value in Sounds.items():
        if key != 'moonlight':
            game_sounds[key] = pygame.mixer.Sound(value)
    return screen, game_images, game_sounds



def main():

    screen, game_images, game_sounds = initGame()

    pygame.mixer.music.load(Sounds['moonlight'])
    pygame.mixer.music.play(-1, 0.0)

    font = pygame.font.Font(None, 24)

    bunny = BunnySprite(image=game_images.get('rabbit'), position=(100, 100))

    acc_record = [0., 0.]

    healthvalue = 194

    arrow_sprites_group = pygame.sprite.Group()

    badguy_sprites_group = pygame.sprite.Group()
    badguy = BadguySprite(game_images.get('badguy'), position=(640, 100))
    badguy_sprites_group.add(badguy)

    badtimer = 100
    badtimer1 = 0

    running, exitcode = True, False
    clock = pygame.time.Clock()
    
    while running:

        screen.fill(0)

        for x in range(SCREENSIZE[0]//game_images['grass'].get_width()+1):
            for y in range(SCREENSIZE[1]//game_images['grass'].get_height()+1):
                screen.blit(game_images['grass'], (x*100, y*100))
        for i in range(4): screen.blit(game_images['castle'], (0, 30+105*i))
 
        countdown_text = font.render(str((90000-pygame.time.get_ticks())//60000)+":"+str((90000-pygame.time.get_ticks())//1000%60).zfill(2), True, (0, 0, 0))
        countdown_rect = countdown_text.get_rect()
        countdown_rect.topright = [635, 5]
        screen.blit(countdown_text, countdown_rect)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                game_sounds['shoot'].play()
                acc_record[1] += 1
                mouse_pos = pygame.mouse.get_pos()
                angle = math.atan2(mouse_pos[1]-(bunny.rotated_position[1]+32), mouse_pos[0]-(bunny.rotated_position[0]+26))
                arrow = ArrowSprite(game_images.get('arrow'), (angle, bunny.rotated_position[0]+32, bunny.rotated_position[1]+26))
                arrow_sprites_group.add(arrow)
                
        key_pressed = pygame.key.get_pressed()
        
        if key_pressed[pygame.K_w]:
            bunny.move(SCREENSIZE, 'up')
        elif key_pressed[pygame.K_s]:
            bunny.move(SCREENSIZE, 'down')
        elif key_pressed[pygame.K_a]:
            bunny.move(SCREENSIZE, 'left')
        elif key_pressed[pygame.K_d]:
            bunny.move(SCREENSIZE, 'right')
        
        for arrow in arrow_sprites_group:
            if arrow.update(SCREENSIZE):
                arrow_sprites_group.remove(arrow)
        
        if badtimer == 0:
            badguy = BadguySprite(game_images.get('badguy'), position=(640, random.randint(50, 430)))
            badguy_sprites_group.add(badguy)
            badtimer=100-(badtimer1*2)
            badtimer1 = 20 if badtimer1>=20 else badtimer1+2
        badtimer -= 1
        
        for badguy in badguy_sprites_group:
            if badguy.update():
                game_sounds['hit'].play()
                healthvalue -= random.randint(4, 8)
                badguy_sprites_group.remove(badguy)
        
        for arrow in arrow_sprites_group:
            for badguy in badguy_sprites_group:
                if pygame.sprite.collide_mask(arrow, badguy):
                    game_sounds['enemy'].play()
                    arrow_sprites_group.remove(arrow)
                    badguy_sprites_group.remove(badguy)
                    acc_record[0] += 1
        
        arrow_sprites_group.draw(screen)
        badguy_sprites_group.draw(screen)
        
        bunny.draw(screen, pygame.mouse.get_pos())
                
        screen.blit(game_images.get('healthbar'), (5, 5))
        
        for i in range(healthvalue):
            screen.blit(game_images.get('health'), (i+8, 8))
        
        if pygame.time.get_ticks() >= 90000:
            running, exitcode = False, True
        if healthvalue <= 0:
            running, exitcode = False, False
        
        pygame.display.flip()
        clock.tick(FPS)
        
        accuracy = acc_record[0]/acc_record[1]*100 if acc_record[1] >0 else 0
        accuracy = '%.2f' % accuracy
    
    
if __name__ == '__main__':
    main()
    

pygame 2.1.0 (SDL 2.0.16, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\cti110016\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
